In [2]:
import openmeteo_requests
import pandas as pd
from retry_requests import retry
import requests
from datetime import datetime
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [3]:
retry_session = retry(requests.Session(), retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [4]:
today = datetime.now().strftime("%Y-%m-%d")
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [38.8951, 51.5085, 35.6895, 21.0245],
	"longitude": [-77.0364, -0.1257, 139.6917, 105.8412],
	"start_date": "2020-01-01",
	"end_date": today,
	"daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"],
	"hourly": ["temperature_2m", "precipitation", "weather_code", "relative_humidity_2m", "wind_speed_10m"]
}
responses = openmeteo.weather_api(url, params=params)

In [5]:
city = ["Washington", "London", "Tokyo", "HaNoi"]
all_data = []
for idx, response in enumerate(responses):
    Latitude = response.Latitude()
    Longitude = response.Longitude()
    Elevation = response.Elevation()
    data = {
        "City": city[idx],
        "Latitude": Latitude,
        "Longitude": Longitude,
        "Elevation": Elevation
    }    
    all_data.append(data)

df = pd.DataFrame(all_data)
df

,City,Latitude,Longitude,Elevation
0,Washington,38.910366,-77.072510,12.0
1,London,51.493847,-0.163025,23.0
2,Tokyo,35.676624,139.691116,40.0
3,HaNoi,21.054480,105.807106,10.0


In [6]:
all_hourly_data = []
for idx, response in enumerate(responses):
	hourly = response.Hourly()
	hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
	hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
	hourly_weather_code = hourly.Variables(2).ValuesAsNumpy()
	hourly_relative_humidity_2m = hourly.Variables(3).ValuesAsNumpy()
	hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()

	hourly_data = {"date": pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
		end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}

	hourly_data["city"] = city[idx]
	hourly_data["temperature_2m"] = hourly_temperature_2m
	hourly_data["precipitation"] = hourly_precipitation
	hourly_data["weather_code"] = hourly_weather_code
	hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
	hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

	df = pd.DataFrame(hourly_data)
	all_hourly_data.append(df)

hourly_df = pd.concat(all_hourly_data)
hourly_df.head()

,date,city,temperature_2m,precipitation,weather_code,relative_humidity_2m,wind_speed_10m
0,2020-01-01 00:00:00+00:00,Washington,7.45,0.0,3.0,55.246738,9.726665
1,2020-01-01 01:00:00+00:00,Washington,7.25,0.0,3.0,53.597992,8.699793
2,2020-01-01 02:00:00+00:00,Washington,6.75,0.0,3.0,49.277168,12.287555
3,2020-01-01 03:00:00+00:00,Washington,6.45,0.0,3.0,49.375435,18.864824
4,2020-01-01 04:00:00+00:00,Washington,6.15,0.0,3.0,54.093021,17.826363


In [7]:
hourly_df.describe()

,temperature_2m,precipitation,weather_code,relative_humidity_2m,wind_speed_10m
count,213024.000000,213024.000000,213024.000000,213024.000000,213024.000000
mean,16.433207,0.168956,11.745358,74.698326,11.973900
std,9.161244,0.800075,20.710222,17.285433,7.249373
min,-15.100000,0.000000,0.000000,10.412831,0.000000
25%,9.350000,0.000000,0.000000,62.801540,6.924738
50%,16.750000,0.000000,3.000000,78.186874,10.399576
75%,23.950001,0.000000,3.000000,89.116516,15.281989
max,41.150002,29.000000,75.000000,100.000000,76.049515


In [8]:
hourly_df.isnull().sum()

date                    0
city                    0
temperature_2m          0
precipitation           0
weather_code            0
relative_humidity_2m    0
wind_speed_10m          0
dtype: int64

In [20]:
weather_description = {
    0: "Clear", 1: "Mostly Clear", 3: "Overcast",
    51: "Drizzle", 61: "Rain", 71: "Snow", 95: 'Thunderstorm'
}
hourly_df['weather'] = hourly_df['weather_code'].map(weather_description)
hourly_df

,date,city,temperature_2m,precipitation,weather_code,relative_humidity_2m,wind_speed_10m,weather
0,2020-01-01 00:00:00+00:00,Washington,7.45,0.0,3.0,55.246738,9.726665,Overcast
1,2020-01-01 01:00:00+00:00,Washington,7.25,0.0,3.0,53.597992,8.699793,Overcast
2,2020-01-01 02:00:00+00:00,Washington,6.75,0.0,3.0,49.277168,12.287555,Overcast
3,2020-01-01 03:00:00+00:00,Washington,6.45,0.0,3.0,49.375435,18.864824,Overcast
4,2020-01-01 04:00:00+00:00,Washington,6.15,0.0,3.0,54.093021,17.826363,Overcast
...,...,...,...,...,...,...,...,...
53251,2026-01-27 19:00:00+00:00,HaNoi,15.65,0.0,1.0,93.474251,2.421652,Mostly Clear
53252,2026-01-27 20:00:00+00:00,HaNoi,15.55,0.0,0.0,94.378685,4.669604,Clear
53253,2026-01-27 21:00:00+00:00,HaNoi,15.25,0.0,0.0,94.671471,4.669604,Clear
53254,2026-01-27 22:00:00+00:00,HaNoi,15.10,0.0,0.0,93.447273,4.792870,Clear


In [10]:
all_daily_data = []
for idx, response in enumerate(responses):
	daily = response.Daily()
	daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
	daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
	daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()

	daily_data = {"date": pd.date_range(
		start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
		end =  pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = daily.Interval()),
		inclusive = "left"
	)}

	daily_data['city'] = city[idx]
	daily_data["temperature_2m_max"] = daily_temperature_2m_max
	daily_data["temperature_2m_min"] = daily_temperature_2m_min
	daily_data["precipitation_sum"] = daily_precipitation_sum

	df = pd.DataFrame(daily_data)
	all_daily_data.append(df)

daily_df = pd.concat(all_daily_data)
daily_df.head()

,date,city,temperature_2m_max,temperature_2m_min,precipitation_sum
0,2020-01-01 00:00:00+00:00,Washington,7.70,2.80,0.0
1,2020-01-02 00:00:00+00:00,Washington,9.20,-1.85,0.0
2,2020-01-03 00:00:00+00:00,Washington,13.30,7.30,6.6
3,2020-01-04 00:00:00+00:00,Washington,14.35,9.10,10.9
4,2020-01-05 00:00:00+00:00,Washington,9.35,0.80,2.1


In [11]:
daily_df.describe()

,temperature_2m_max,temperature_2m_min,precipitation_sum
count,8876.000000,8876.000000,8876.000000
mean,20.775162,12.665930,4.054946
std,8.858594,8.929455,9.860646
min,-8.000000,-15.100000,0.000000
25%,13.550000,5.550000,0.000000
50%,21.250000,12.800000,0.300000
75%,28.150000,20.200001,3.425000
max,41.150002,29.600000,176.799988


In [12]:
daily_df.isnull().sum().sum()

np.int64(0)

In [13]:
# Load environment variables
load_dotenv()
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)  

with engine.connect() as conn:
    hourly_df.to_sql(
        name="weather_hourly",
        con=conn,
        if_exists="replace",
        index=False
    )
    
    daily_df.to_sql(
        name="weather_daily",
        con=conn,
        if_exists="replace",
        index=False
    )
connection_string = (
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}"
    f"@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)
engine = create_engine(connection_string)

hourly_df.to_sql(
    "weather_hourly",
    con=engine,
    if_exists="replace",
    index=False,
    method="multi"
)

daily_df.to_sql(
    "weather_daily",
    con=engine,
    if_exists="replace",
    index=False,
    method="multi"
)

print("Load completed")

Load completed


In [14]:
import pandas as pd
import sqlalchemy

print(pd.__version__)
print(sqlalchemy.__version__)


2.3.3
2.0.45
